# Spectrometer Controller
OceanInsight [Redtide - USB2000](https://www.oceanoptics.com/resources/legacy-spectrometers/) \
[SeaBreeze](https://github.com/ap--/python-seabreeze/tree/main) 

### Load libraries

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os
# For helper functions
from datetime import datetime, timezone
import h5py

Load Spectrometer
- Be sure to run "seabreeze_os_setup" in the terminal inside the conda environment after installing seabreeze
- If you are on wsl, you will have to bind the usb to wsl. This will only work on Windows 11 on WSL2. Linux or Mac highly recommended

Run "usbipd attach --wsl --busid 5-1" in PowerShell

Run "lsusb" in wsl

In [4]:
import seabreeze
from seabreeze.spectrometers import Spectrometer, list_devices
devices = list_devices()
print(devices)
spec = Spectrometer.from_first_available()

[<SeaBreezeDevice USB2000:USB2G50689>]


### Take a spectra

In [6]:
# Set integration time
spec.integration_time_micros(100000)  # 0.1 seconds

# Get wavelengths
wavelengths = spec.wavelengths()

# Get intensities
intensities = spec.intensities()

data = {'Wavelength':wavelengths, 'Intensity':intensities}
spectra = pd.DataFrame(data)
plt.plot(spectra['Wavelength'], spectra['Intensity'])
plt.xlabel('Wavelength')
plt.ylabel('Intensity')

SeaBreezeError: Error: Data transfer error

### Helper functions

In [21]:
def get_spectra(int_time):
    """
    Collects a spectra using the loaded OceanInsight spectrometer with seabreeze

    Parameters:
        int_time (int): Integration time. Amount of time the spectrometer collects data for in microseconds (us)

    Returns:
        data (list): contains two lists
            'Wavelength' (list, floats): wavelengths of spectra in nanometers (nm)
            'Intensity' (list, floats): intensity of light at the returned wavelengths in detector counts (counts)
    """
    spec.integration_time_micros(int_time)  # 0.1 seconds

    wavelengths = spec.wavelengths()
    
    intensities = spec.intensities()
    
    data = [wavelengths, intensities]
    return data

def get_time():
    """
    returns the date and time in UTC format YYYY-MM-DDThh:mm:ssZ
    Uses datetime library
    """
    return datetime.now(timezone.utc).isoformat()

def write_specs_hdf5(filename, group_name, spectra_list, time_list, wavelengths, metadata):
    """
    Write time-indexed spectrometer data to an HDF5 file using a structured,
    chunked, and compressed storage layout.

    This function stores spectral acquisition data in a hierarchical HDF5 format
    optimized for large datasets, fast I/O, and downstream scientific analysis.
    Data are written into a dedicated group (e.g., "dark", "fast") containing
    numerical datasets and associated experimental metadata.

    Parameters
    ----------
    filename : str
        Path to the HDF5 file. The file will be created if it does not exist.
        If the group already exists, it will be overwritten.

    group_name : str
        Name of the HDF5 group under which the data will be stored
        (e.g., "dark", "fast", "calibration", "sample_run").

    spectra_list : list of np.ndarray
        List of 1D NumPy arrays containing spectral intensity values.
        Each array must have identical length corresponding to the number
        of detector pixels or wavelength bins.
        Shape after stacking: (N_spectra, N_pixels).

    time_list : list or np.ndarray
        List of acquisition timestamps corresponding to each spectrum.
        Must be the same length as `spectra_list`.
        Stored as float64 for precision timing applications.

    wavelengths : np.ndarray
        1D array of wavelength values (in nm or instrument units) corresponding
        to spectral channels/pixels. Length must match the second dimension
        of the stacked spectra array.

    metadata : dict
        Dictionary of experimental and instrument metadata to be stored
        as HDF5 attributes. Typical entries include:
            - integration_time_us
            - dist_mm
            - instrument_id
            - acquisition_mode
            - operator
            - experiment_id
            - calibration_version

    Storage Layout
    --------------
    /<group_name>/
        spectra      : float32 [N_spectra, N_pixels]  (chunked, compressed)
        timestamps   : float64 [N_spectra]            (chunked, compressed)
        wavelengths  : float32 [N_pixels]
        attrs        : metadata fields

    Notes
    -----
    - Uses gzip compression and chunked datasets for efficient storage and I/O.
    - Optimized for large-scale spectral acquisition workflows.
    - Compatible with NumPy, h5py, xarray, Dask, and scientific data pipelines.
    - Designed for reproducibility and long-term data archival.
    - Supports hierarchical separation of acquisition modes (e.g., dark/fast).

    Raises
    ------
    ValueError
        If input array lengths or shapes are inconsistent.
    IOError
        If file cannot be created or written.

    """
    with h5py.File(filename, "a") as f:
        if group_name in f:
            del f[group_name]  # overwrite safely
        
        g = f.create_group(group_name)

        spectra_arr = np.stack(spectra_list)  # shape: (N, n_pixels)
        time_arr = np.array(time_list)

        g.create_dataset(
            "spectra",
            data=spectra_arr,
            compression="gzip",
            chunks=True,
            dtype="float32"
        )

        g.create_dataset(
            "timestamps",
            data=time_arr,
            compression="gzip",
            chunks=True,
            dtype="float64"
        )

        g.create_dataset(
            "wavelengths",
            data=wavelengths,
            dtype="float32"
        )

        # metadata
        for k, v in metadata.items():
            g.attrs[k] = v

## Experiment Prep
Before adding the ion solution to top of gel
>Measure the distance from the top of gel to the point of interest\
>Confirm integration time is reasonable\
>Measure with light entirely blocked, full integration time (dark current correction)\
>Measure with light entirely blocked, fast integration time (instrument noise correction)\
>Measure background (sample with no ions)

After adding the ion solution to top of gel
>Begin measurement script\
>Confirm writing to file

In [ ]:
# Experiment Prep
# Test Spectra for integration time
dist = 5 #mm from top of gel
int_time_s = 10 #seconds

int_time = np.multiply(int_time_s, 1E6) #microseconds

#Take spectra
spectra = get_spectra(int_time)
time = get_time().split('.')[0]

data = {'Wavelength':spectra[0], 'Intensity':spectra[1]}
spectra = pd.DataFrame(data)
plt.plot(spectra['Wavelength'], spectra['Intensity'])
plt.title(f'Spectra at {time}')
plt.xlabel('Wavelength')
plt.ylabel('Intensity')

### Create Experiment Folder

In [33]:
Experiment_Name = 'Experiment_0'
Exp_folder_path = Path(os.getcwd() + '/' + Experiment_Name)
print(Exp_folder_path)
Exp_folder_path.mkdir(parents=True, exist_ok=True)

/home/zrwylie/UVVisScan/Scanning-UVVis-4mmNMRTube/Experiment_0


In [13]:
#Cover instrument sensor
spec = Spectrometer.from_first_available()

dist = 5 #mm from top of gel
int_time_s = 10 #seconds

# Number of dark and fast spectra
dark_spec_count = 10
fast_spec_count = 10

int_time = np.multiply(int_time_s, 1E6) #microseconds
min_int_time = 10 #microseconds

dark_spectra = []
dark_times = []

for i in range(dark_spec_count):
    spec.integration_time_micros(int_time)
    intensity = spec.intensities()
    time = get_time()
    dark_spectra.append(intensity)
    dark_times.append(time)

fast_spectra = []
fast_times = []

for i in range(fast_spec_count):
    spec.integration_time_micros(int_time)
    intensity = spec.intensities()
    time = get_time()
    fast_spectra.append(intensity)
    fast_times.append(time)

wavelengths = spec.wavelengths() # from spectrometer

# Write dark spectra
write_specs_hdf5(
    "SpectrometerCalibration.h5",
    "DarkSpectra",
    dark_spectra,
    dark_times,
    wavelengths,
    metadata={
        "integration_time_us": int_time,
        "dist_mm": dist,
        "type": "dark"
    }
)

# Write fast spectra
write_specs_hdf5(
    "SpectrometerCalibration.h5",
    "FastSpectra",
    fast_spectra,
    fast_times,
    wavelengths,
    metadata={
        "integration_time_us": min_int_time,
        "dist_mm": dist,
        "type": "fast"
    }
)

'2026-01-27T00:41:08'

0
1
2
3
4
5
6
7
8
9
